# Implementation of Selecting Comprehensive set of Reviews

In [ ]:
import pandas as pd
import gzip

import nltk
import nltk.data
import nltk.tokenize
from nltk.tree import Tree

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

### Read Data from JSON file

In [ ]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Musical_Instruments.json.gz')

##### 1) only 4 columns are required reviewtext, asin, helpful (calculated quality score) and overall
##### 2) prune items for which reviews are less than 20 or addition of positive and negative helpful votes are less than 10

In [ ]:
count = 1
initValue = 0 
review = []
sizeByGroup = []
setOfReviews = []

for i in range(1, len(df)):
    if df.asin[i-1] == df.asin[i]:
        count += 1
    else :
        sizeByGroup.append(count)
        count = 1
        
for i in range(len(sizeByGroup)):
    if sizeByGroup[i] > 20:
        for j in range(sizeByGroup[i]):
            if sum(df['helpful'].iloc[initValue+j]) > 10:
                review.append(df['reviewText'].iloc[initValue+j])
                setOfReviews.append([df['asin'].iloc[initValue+j], df['reviewText'].iloc[initValue+j], (df['helpful'].iloc[initValue+j])[0]/sum(df['helpful'].iloc[initValue+j]), df['overall'].iloc[initValue+j]])
    initValue += sizeByGroup[i]
    
refinedSetOfReviews = pd.DataFrame(setOfReviews, columns=['asin', 'reviewText', 'helpful', 'overall'])

## Two methods have been tried for collection of noun and noun phrases
###### *use one of them

### 1) Stanford NLP
#### Run the server using all jars in the current directory (e.g., the CoreNLP home directory)
###### java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 15000

In [ ]:
from pycorenlp import StanfordCoreNLP
nlp = StanfordCoreNLP('http://localhost:9000')

In [ ]:
def find_noun_phrases(tree):
    return [subtree for subtree in tree.subtrees(lambda t: t.label()=='NN')]

def find_head_of_np(np):
    noun_tags = ['NN', 'NNS', 'NNP', 'NNPS']
    top_level_trees = [np[i] for i in range(len(np)) if type(np[i]) is Tree]
    top_level_nouns = [t for t in top_level_trees if t.label() in noun_tags]
    if len(top_level_nouns) > 0:
        return top_level_nouns[-1][0]
    else:
        top_level_nps = [t for t in top_level_trees if t.label()=='NP']
        if len(top_level_nps) > 0:
            return find_head_of_np(top_level_nps[-1])
        else:
            nouns = [p[0] for p in np.pos() if p[1] in noun_tags]
            if len(nouns) > 0:
                return nouns[-1]
            else:
                return np.leaves()[-1]

### 2) nltk pos_tag

In [ ]:
L = []
t = []
L1 = []
count = 0
ps = PorterStemmer()
stop_words = {'k', 'particularly', 'normally', 'himself', 'unto', 'herself', 'followed', 'are', 'latter', 'tried', 'it', 'hers', 'whence', 'none', 'become', 'besides', 'lest', 'about', 'able', 'concerning', 'seems', 'she', 'more', 'as', 'wonder', 'o', 'beside', 'because', 'seriously', 'will', 'said', 'everything', 'associated', 'took', 'thorough', 'let', 'specified', 'gotten', 'no', 'one', 'howbeit', 'us', 'further', 'eg', 'ought', 'anyways', 'nearly', 'consider', 'happens', 'several', 'whereby', 'merely', 'but', 'want', 'second', 'downwards', 'whereupon', 'inner', 'even', 'a', 'yes', 'next', 'vs', 'so', 'respectively', 'nobody', 'would', 'upon', 'nothing', 'particular', 'at', 'meanwhile', 'clearly', 'moreover', 'soon', 'nine', 'sub', 'value', 'indeed', 'without', 'think', 'immediate', 'saw', 'provides', 'sorry', 'contain', 'sent', 'seem', 'needs', 'less', 'done', 'maybe', 'their', 'within', 'selves', 'nevertheless', 'near', 'un', 'together', 'exactly', 'p', 'need', 'saying', 'own', 'following', 'everyone', 'against', 'formerly', 'the', 'right', 'consequently', 'far', 'do', 'aside', 'his', 'unlikely', 'seeming', 'nd', 'five', 'again', 'where', 'q', 'kept', 'please', 'being', 'behind', 'considering', 'greetings', 'inc', 'and', 'certain', 'cause', 'gives', 'que', 'course', 'used', 'appropriate', 'tends', 're', 'to', 'thereby', 'd', 'per', 'neither', 'nor', 'c', 'b', 'my', 's', 'ever', 'indicated', 'whose', 'l', 'hardly', 'going', 'reasonably', 'only', 'well', 'someone', 'comes', 'almost', 'we', 'four', 'way', 'else', 'thence', 'toward', 'beforehand', 'yourself', 'very', 'was', 'goes', 'probably', 'keep', 'three', 'which', 'various', 'having', 'whenever', 'not', 'instead', 'novel', 'obviously', 'available', 'might', 'trying', 'taken', 'first', 'furthermore', 'go', 'insofar', 'third', 'these', 'cant', 'anything', 'x', 'came', 'unfortunately', 'herein', 'whoever', 'zero', 'according', 'while', 'two', 'gone', 'away', 'could', 'seemed', 'such', 'over', 'its', 'under', 'others', 'mainly', 'until', 'much', 'six', 'were', 'should', 'take', 'quite', 'few', 'r', 'throughout', 'if', 'thanks', 'anyone', 'seeing', 'whereas', 'ltd', 'whether', 'certainly', 'therefore', 'somewhat', 'you', 'says', 'try', 'when', 'follows', 'serious', 'know', 'lately', 'got', 'out', 'relatively', 'etc', 'does', 'ex', 'help', 'getting', 'for', 'wherein', 'becoming', 'tell', 'twice', 'e', 'itself', 'really', 'theres', 'theirs', 'thoroughly', 'presumably', 'rd', 'edu', 'hereby', 'towards', 'name', 'however', 'noone', 'any', 'v', 'placed', 'changes', 'hello', 'like', 'f', 'liked', 'ignored', 'thus', 'specifying', 'both', 'all', 'specify', 'then', 'sometime', 'by', 'necessary', 'yours', 'indicate', 'definitely', 'whither', 'possible', 'here', 'whatever', 'between', 'otherwise', 'everybody', 'other', 'latterly', 'is', 'come', 'up', 'through', 'nowhere', 'in', 'sometimes', 'thereupon', 'especially', 'hopefully', 'use', 'least', 'allow', 'eight', 'whom', 'truly', 'me', 'via', 'this', 'on', 'onto', 'allows', 'below', 'asking', 'uses', 'often', 'new', 'elsewhere', 'always', 'inward', 'down', 'apart', 'thank', 'than', 'better', 'must', 'old', 'have', 'becomes', 'each', 'went', 'before', 'already', 'th', 'myself', 'w', 'sure', 'with', 'containing', 'g', 'com', 'little', 'believe', 'did', 'either', 'same', 'look', 'okay', 'beyond', 'cannot', 'uucp', 'thats', 'still', 'namely', 'useful', 'somewhere', 'ourselves', 'ones', 'from', 'those', 'somebody', 'actually', 'an', 'looking', 'brief', 'therein', 'shall', 'knows', 'ok', 'her', 'can', 'despite', 'yet', 'your', 'though', 'h', 't', 'afterwards', 'self', 'fifth', 'looks', 'et', 'seen', 'overall', 'known', 'some', 'entirely', 'mean', 'doing', 'qv', 'anywhere', 'seven', 'sensible', 'regardless', 'regards', 'accordingly', 'across', 'ie', 'forth', 'amongst', 'who', 'alone', 'since', 'awfully', 'z', 'n', 'contains', 'anybody', 'hither', 'unless', 'he', 'plus', 'every', 'likely', 'along', 'them', 'u', 'am', 'our', 'hereafter', 'once', 'described', 'of', 'oh', 'tries', 'appear', 'keeps', 'him', 'm', 'usually', 'something', 'somehow', 'above', 'too', 'there', 'perhaps', 'around', 'now', 'later', 'had', 'enough', 'why', 'example', 'been', 'except', 'thereafter', 'see', 'say', 'themselves', 'i', 'sup', 'thanx', 'anyway', 'never', 'former', 'hereupon', 'causes', 'outside', 'during', 'appreciate', 'or', 'using', 'ours', 'everywhere', 'wants', 'became', 'most', 'j', 'be', 'how', 'mostly', 'anyhow', 'viz', 'among', 'hence', 'inasmuch', 'last', 'whereafter', 'indicates', 'rather', 'just', 'gets', 'they', 'after', 'may', 'get', 'co', 'given', 'another', 'ask', 'has', 'although', 'different', 'wish', 'thru', 'also', 'whole', 'non', 'best', 'what', 'off', 'yourselves', 'y', 'wherever', 'into', 'that', 'many', 'currently', 'hi', 'welcome', 'secondly', 'corresponding', 'regarding', 'willing'}
for i in range(len(refinedSetOfReviews)):
    raw = refinedSetOfReviews.reviewText[i]
    raw.replace(".",". ")
    tokenized_sentences = nltk.sent_tokenize(raw)
    L2 = []
    count += len(tokenized_sentences)
    for j in range(len(tokenized_sentences)): 
        text = nltk.word_tokenize((tokenized_sentences[j]))
        pos_sentences = nltk.pos_tag(text)      
        L3 = []
        for k in range(len(pos_sentences)):
            if pos_sentences[k][1] == "NN":
                if (pos_sentences[k][0]).lower() not in stop_words:
                    L3.append(pos_sentences[k][0].lower())
        if L3:
            L.append(list(set(L3)))
            L2.append(list(set(L3)))
            t.append(tuple(set(L3)))
    L1.append(L2)

### Finding Fequent Item Set

In [ ]:
from pymining import itemmining
transactions = tuple(t)
relim_input = itemmining.get_relim_input(transactions)
report = itemmining.relim(relim_input, min_support=1000)

In [ ]:
listReport = list(report)
attributes = []
for i in listReport:
    conv = list(i)
    temp = []
    for j in conv:
        temp.append(j)
    attributes.append(temp)

## Seven Coverage Functions are:

### 1) Unit Coverage function f_{u}

In [ ]:
def f_u(S, a):
    if len(S) >= 1:
        for s in S:
            if len(a) == 1:
                if s.count(a[0]) > 0:
                    return 1
            else:
                flag = 0
                for a1 in a:
                    if s.count(a1) > 0:
                        flag += 1
                if flag == len(a):
                    return 1
        return 0
    else:
        return 0   

### 2) Quality Coverage function f_{q}

In [ ]:
def f_q(S, a):
    maxQualityScore = 0
    for s in S:
        flag = 0
        for a1 in a:
            if s.count(a1) == 0:
                flag = 1
                break
        if flag == 0:
            ind = review.index(s)
            if maxQualityScore < refinedSetOfReviews.helpful[ind]:
                maxQualityScore = refinedSetOfReviews.helpful[ind]
    return maxQualityScore

### 3) Soft  Unit Coverage function f_{su}

In [ ]:
def f_su(S, a):
    S1 = []
    S2 = []
    for s in S:
        if refinedSetOfReviews.overall[review.index(s)] > 3:
            S1.append(s)
        else:
            S2.append(s)
    return (f_u(S1, a) + f_u(S2, a))

### 4) Soft Quality Coverage function f_{sq}

In [ ]:
def f_sq(S, a):
    S1 = []
    S2 = []
    for s in S:
        if refinedSetOfReviews.overall[review.index(s)] > 3:
            S1.append(s)
        else:
            S2.append(s)
    return (f_q(S1, a) + f_q(S2, a))

### 5) Group Unit Coverage function f_{gu}

In [ ]:
def f_gu(S, a):
    S1 = []
    S2 = []
    for s in S:
        if refinedSetOfReviews.overall[review.index(s)] > 3:
            S1.append(s)
        else:
            S2.append(s)
    return min(f_u(S1, a), f_u(S2, a))

### 6) Group Quality Coverage function f_{gq}

In [ ]:
def f_gq(S, a):
    S1 = []
    S2 = []
    for s in S:
        if refinedSetOfReviews.overall[review.index(s)] > 3:
            S1.append(s)
        else:
            S2.append(s)
    return min(f_q(S1, a), f_q(S2, a))

### 7) Quality Metric

In [ ]:
def fQuality(S):
    score = 0
    for s in S:
        score += refinedSetOfReviews.helpful[review.index(s)]
    return score/len(S)

## Six Cumualtive Fucntions are:

### 1) Unit Cumulative Function F_{u} 

In [ ]:
def F_u(S):
    score = 0
    for a in attributes:
        score += f_u(S, a)
    return score

### 2) Quality Cumulative Function F_{q}

In [ ]:
def F_q(S):
    score = 0
    for a in attributes:
        score += f_q(S, a)
    return score

### 3) Soft Unit Cumulative Function F_{su}

In [ ]:
def F_su(S):
    score = 0
    for a in attributes:
        score += f_su(S, a)
    return score

### 4) Soft Quality Cumulative Function F_{sq}

In [ ]:
def F_sq(S):
    score = 0
    for a in attributes:
        score += f_sq(S, a)
    return score

### 5) Group Unit Cumulative Function F_{gu}

In [ ]:
def F_gu(S):
    score = 0
    for a in attributes:
        score += f_gu(S, a)
    return score

### 6) Group Quality Cumulative Function F_{gq}

In [ ]:
def F_gq(S):
    score = 0
    for a in attributes:
        score += f_gq(S, a)
    return score

## First Greedy Algorithm

### First Greedy Algorithm for F_{u}

In [ ]:
def greedyAlgoF_u(k):   
    Sout = []
    flag = [0]*len(review)
    for i in range(0,k):
        maxDeltaS = 0
        maxDeltaIndex = -1
        for r in review:
            if flag[review.index(r)] == 0:
                Sdummy = list(Sout)
                Sdummy.append(r)
                deltaS = F_u(Sdummy) - F_u(Sout)
                if deltaS > maxDeltaS:
                    maxDeltaS = deltaS
                    maxDeltaIndex = review.index(r)
        if maxDeltaIndex != -1:
            Sout.append(review[maxDeltaIndex])
#             print(maxDeltaIndex)
            flag[maxDeltaIndex] = 1
    return Sout

In [ ]:
k = 5
greedyAlgoF_u(k)

### First Greedy Algorithm for F_{q}

In [ ]:
def greedyAlgoF_q(k):   
    Sout = []
    flag = [0]*len(review)
    for i in range(0,k):
        maxDeltaS = 0
        maxDeltaIndex = -1
        for r in review:
            if flag[review.index(r)] == 0:
                Sdummy = list(Sout)
                Sdummy.append(r)
                deltaS = F_q(Sdummy) - F_q(Sout)
                if deltaS > maxDeltaS:
                    maxDeltaS = deltaS
                    maxDeltaIndex = review.index(r)
        if maxDeltaIndex != -1:
            Sout.append(review[maxDeltaIndex])
#             print(maxDeltaIndex)
            flag[maxDeltaIndex] = 1
    return Sout

In [ ]:
k = 5
greedyAlgoF_q(k)

### First Greedy Algorithm for F_{su}

In [ ]:
def greedyAlgoF_su(k):   
    Sout = []
    flag = [0]*len(review)
    for i in range(0,k):
        maxDeltaS = 0
        maxDeltaIndex = -1
        for r in review:
            if flag[review.index(r)] == 0:
                Sdummy = list(Sout)
                Sdummy.append(r)
                deltaS = F_su(Sdummy) - F_su(Sout)
                if deltaS > maxDeltaS:
                    maxDeltaS = deltaS
                    maxDeltaIndex = review.index(r)
        if maxDeltaIndex != -1:
            Sout.append(review[maxDeltaIndex])
#             print(maxDeltaIndex)
            flag[maxDeltaIndex] = 1
    return Sout

In [ ]:
k = 5
greedyAlgoF_su(k)

### First Greedy Algorithm for F_{sq}

In [ ]:
def greedyAlgoF_sq(k):   
    Sout = []
    flag = [0]*len(review)
    for i in range(0,k):
        maxDeltaS = 0
        maxDeltaIndex = -1
        for r in review:
            if flag[review.index(r)] == 0:
                Sdummy = list(Sout)
                Sdummy.append(r)
                deltaS = F_sq(Sdummy) - F_sq(Sout)
                if deltaS > maxDeltaS:
                    maxDeltaS = deltaS
                    maxDeltaIndex = review.index(r)
        if maxDeltaIndex != -1:
            Sout.append(review[maxDeltaIndex])
#             print(maxDeltaIndex)
            flag[maxDeltaIndex] = 1
    return Sout

In [ ]:
k = 5
greedyAlgoF_sq(k)

## Second t-Greedy Algorithm

In [ ]:
import itertools
R1 = []
R2 = []
for r in range(len(review)):
    if refinedSetOfReviews.overall[r] > 3:
        R1.append(review[r])
    else:
        R2.append(review[r])
T = list(itertools.product(R1, R2))

In [ ]:
def Ps(t, S):
    counter = 0
    for a in attributes:
        if len(a) > 1:
            if (t[0].count(a[0]) > 0 and t[1].count(a[0]) == 0) and (t[0].count(a[1]) > 0 and t[1].count(a[1]) == 0):
                for s in S:
                    sflag = 0
                    for a1 in a:
                        if s.count(a1) == 0:
                            sflag = 1
                    if sflag == 0:
                        counter += 1
            elif (t[1].count(a[0]) > 0 and t[0].count(a[0]) == 0) and (t[1].count(a[1]) > 0 and t[0].count(a[1]) == 0):
                for s in S:
                    sflag = 0
                    for a1 in a:
                        if s.count(a1) == 0:
                            sflag = 1
                    if sflag == 0:
                        counter += 1
        else:
            if (t[0].count(a[0]) > 0 and t[1].count(a[0]) == 0) or (t[0].count(a[0]) == 0 and t[1].count(a[0]) > 0):
                for s in S:
                    if a[0] not in s:
                        counter += 1
    return counter

### Second t-greedy algorithm for F_{gu}

In [ ]:
def t_greedyAlgoF_gu(k): 
    sOut = []
    flag = [0]*len(T)
    while len(sOut) < k:
        b = k - len(sOut)
        maxCs = 0
        maxPs = 0
        maxDeltaSCs = 0
        maxDeltaIndex = -1
        for t in range(0, len(T)):
            Cs = 0
            if flag[t] == 0:
                Sdummy = list(sOut)
                Sdummy.append(T[t][0])
                Sdummy.append(T[t][1])
                deltaS = F_gu(Sdummy) - F_gu(sOut)
                if T[t][0] not in sOut:
                    Cs += 1
                if T[t][1] not in sOut:
                    Cs += 1
                if Cs <= b and Cs != 0:
                    if deltaS/Cs > maxDeltaSCs:
                        maxDeltaSCs = deltaS/Cs
                        maxDeltaIndex = t
                    elif deltaS/Cs == maxDeltaSCs:
                        if Ps(T[t], sOut) > maxPs:
                            maxPs = Ps(T[t], sOut)
                            maxDeltaSCs = deltaS/Cs
                            maxDeltaIndex = t
        if maxDeltaIndex != -1:
            if T[maxDeltaIndex][0] not in sOut:
                sOut.append(T[maxDeltaIndex][0])
            if T[maxDeltaIndex][1] not in sOut:
                sOut.append(T[maxDeltaIndex][1])
            print(maxDeltaIndex)
            flag[maxDeltaIndex] = 1
    return sOut

In [ ]:
k = 5
t_greedyAlgoF_gu(k)

### Second t-greedy algorithm for F_{gq}

In [ ]:
def t_greedyAlgoF_gq(k): 
    sOut = []
    flag = [0]*len(T)
    while len(sOut) < k:
        b = k - len(sOut)
        maxCs = 0
        maxPs = 0
        maxDeltaSCs = 0
        maxDeltaIndex = -1
        for t in range(0, len(T)):
            Cs = 0
            if flag[t] == 0:
                Sdummy = list(sOut)
                Sdummy.append(T[t][0])
                Sdummy.append(T[t][1])
#                 print(Sdummy)
                deltaS = F_gq(Sdummy) - F_gq(sOut)
                if T[t][0] not in sOut:
                    Cs += 1
                if T[t][1] not in sOut:
                    Cs += 1
                if Cs <= b and Cs != 0:
                    if deltaS/Cs > maxDeltaSCs:
                        maxDeltaSCs = deltaS/Cs
                        maxDeltaIndex = t
                    elif deltaS/Cs == maxDeltaSCs:
                        if Ps(T[t], sOut) > maxPs:
                            maxPs = Ps(T[t], sOut)
                            maxDeltaSCs = deltaS/Cs
                            maxDeltaIndex = t
        if maxDeltaIndex != -1:
            if T[maxDeltaIndex][0] not in sOut:
                sOut.append(T[maxDeltaIndex][0])
            if T[maxDeltaIndex][1] not in sOut:
                sOut.append(T[maxDeltaIndex][1])
            print(maxDeltaIndex)
            flag[maxDeltaIndex] = 1
    return sOut

In [ ]:
k = 5
t_greedyAlgoF_gq(k)

## Three Baseline Algorithms are:

### Top Quality  Algorithm

In [ ]:
def top_quality(k):
    topQuality = []
    flag = [0]*len(review)
    for i in range(0, k):
        maxscore = 0
        maxindex = -1
        for r in range(len(review)):  
            if flag[r] == 0 and refinedSetOfReviews.helpful[r] > maxscore:
                maxscore = refinedSetOfReviews.helpful[r]
                maxindex = r
        flag[maxindex] = 1        
        if maxindex != -1:
            topQuality.append(review[maxindex])
    return topQuality


In [ ]:
k = 5
top_quality(k)

### Top Length Algorithm

In [ ]:
def top_len(k):  
    toplength = []
    flag = [0]*len(review)
    for i in range(0, k):
        maxlen = 0
        maxindex = -1
        for r in range(len(review)):  
            if flag[r] == 0 and len(refinedSetOfReviews.reviewText[r]) > maxlen:
                maxlen = len(refinedSetOfReviews.reviewText[r])
                maxindex = r
        flag[maxindex] = 1        
        if maxindex != -1:
            toplength.append(review[maxindex])
    return toplength

In [ ]:
k = 5
top_len(k)

### Random Algorithm

In [ ]:
from random import randint
def fRandom(k):
    Sreview = []
    for i in range(0, k):
        Sreview.append(review[randint(0,len(review)-1)])
    return Sreview

In [ ]:
k = 5
fRandom(k)

#### Iterate 1000 times for getting average score for random algorithm
#### And calculate it for various Cumulative Coverage Score Functions by changing  F_gq with others 

In [ ]:
sum_rand = 0
for j in range(0,1000):
    Set = fRandom()
    sum_rand += F_gq(Set)
metric = sum_rand/1000
print(metric)

#### Every Algorithm above gives us a set of five reviews. For this set, calculate six cumulative coverage scores and quality score. Normalized these scores with corresponding scores considering set with all reviews i.e. k = n, respectively.

##### for example, find seven scores on the set obtained by greedyAlgoF_sq(k) 
##### Normalize the scores by corresponding scores calculated for all reviews

In [ ]:
# e.g.
k = 5
print(F_u(greedyAlgoF_sq(k))/(F_u(review)))
print(F_q(greedyAlgoF_sq(k))/(F_q(review)))
print(F_su(greedyAlgoF_sq(k))/(F_su(review)))
print(F_sq(greedyAlgoF_sq(k))/(F_sq(review)))
print(F_gu(greedyAlgoF_sq(k))/(F_gu(review)))
print(F_gq(greedyAlgoF_sq(k))/(F_gq(review)))
print(fQuality(greedyAlgoF_sq(k))/(fQuality(review)))